# 准备数据

In [1]:
import sys

sys.path.append('../')

from alignment_utils import one_file_per_process
from alignment_utils import extract_and_combine_numbers
from alignment_utils import longest_increasing_subsequence_index
from alignment_utils import find_answer_split_str
from alignment_utils import find_next_question_index
from alignment_utils import refine_answers
from alignment_utils import match_specific_from_end
from alignment_utils import answer_area_str_process
from alignment_utils import generate_answer_area_string
from alignment_utils import align_answers_in_questions

In [2]:
import os
import glob
from pathlib import Path

examination_paper_list = []

path = Path("./answer_markdown")

for file in path.glob("*.md"):
    with open(file, "r", encoding="utf-8") as f:
        examination_paper_list.append({
            "file_path": file,
            "text": one_file_per_process(f.read())
        })
len(examination_paper_list)

1615

In [3]:
import re

def type_of_judgment(text):
    # 拆分成行
    lines = text.splitlines()
    question_number_list = []
    
    pattern = r'^[一二三四五六七八九十][、．.]'
    for i in range(len(lines)):
        line = lines[i]
        match = re.match(pattern, line)
        if match:
#             print(match.group(0)[0])
            if i+1 < len(lines):
                next_line = lines[i+1]
                question_number_list.append(extract_and_combine_numbers(next_line))
    
    # 计算1在数组中出现的次数
    count_of_ones = question_number_list.count(1)
    
    return count_of_ones > len(question_number_list) / 2


In [4]:
type_of_judgment("一、\n1.\n二、\n1.\n2.\n")

True

# 寻找题目

In [5]:
def get_all_question(text):
    # 拆分成行
    lines = text.splitlines()
    
    # 定义不准确的题目列表    
    inaccuracy_question = []
    
    # 从0的位置寻找第一道题
    index = find_next_question_index(0, lines)
    
    while index < len(lines):
        # 寻找下一个题目的index
        next_index = find_next_question_index(index, lines)
        
        inaccuracy_question.append("\n".join(lines[index: next_index]))
        index = next_index
#     for i in inaccuracy_question:
#         print(i)
#         print()
#     print( [extract_and_combine_numbers(topic) for topic in inaccuracy_question])
    # 通过"最长递增子序列"寻找每个精准的题目所在inaccuracy_question对应的下标
    all_question_indexs = longest_increasing_subsequence_index(inaccuracy_question)
    
    # 定义准确的题目列表    
    all_question = []
    # index为all_question_indexs的下标，all_question_indexs[index]为inaccuracy_question的下标
    for index, question_index in enumerate(all_question_indexs):
        if index == len(all_question_indexs) -1 :
            all_question.append("\n".join(inaccuracy_question[question_index:]))
            break
            
        all_question.append("\n".join(inaccuracy_question[question_index:all_question_indexs[index+1]]))
    
    if not all_question:
        return None, None
    
    
    # 尝试寻找用于分割答题区与答案区的字符串，返回值为int/str，如果是str则是分割的字符串
    # 本质是在"all_question[-1]"寻找答案关键字等字样
    answer_split_str = find_answer_split_str(all_question)
    
    if isinstance(answer_split_str, str):
        # 如果找到这个拆分的字符串了，则先把最后一道题的内容进行拆分
        all_question[-1] = all_question[-1].split(answer_split_str)[0]
        
    if text.splitlines()[0] in all_question[-1]:
        answer_split_str = text.splitlines()[0]
        # 看看试卷的title是否出现在"all_question[-1]"位置，如果出现则删除
        all_question[-1] = all_question[-1].replace("\n"+text.splitlines()[0]+"\n","")

    return all_question, answer_split_str

# 创建测试样本

In [6]:
#994 题目为 图片+文字（无法用规则提取）
#1087 一 1.2.3 二 1.2.3 （规则还没写）
#1482 解析错误 暂未解决（规则BUG）

In [44]:
import random
random_number = random.randint(0, len(examination_paper_list))
print(random_number)
test_text = examination_paper_list[random_number]["text"]
# test_text = examination_paper_list[994]["text"]

1482


In [45]:
test_text.splitlines()

['2020－2021学年上期三年级数学期末测试卷',
 '1\\. 口算![](./notebook/image/media/image1.wmf)',
 '86＋16＝ 61－25＝ 36＋37＝ 33＋24－15＝',
 '250＋170＝ 17×3＝ 5×16＝ 280－150＋30＝',
 '123×0＝ 101×5＝',
 '【答案】102，36，73，42',
 '420，51，80，160',
 '0，505，1，',
 '【解析】',
 '【分析】',
 '【详解】略',
 '二、用心思考，准确填写。',
 '2\\. 用分数表示涂色部分，并在横线上填写"＞""＝"或"＜"。',
 '![](./notebook/image/media/image5.png)',
 '[ ]{.underline} （ ）',
 '【答案】＞；',
 '【解析】',
 '【分析】把15个小蘑菇平均分成5份，1份是3个，涂颜色的是3个正好是1份，是总数。分子相同时，分母小的分数大。',
 '【详解】![](./notebook/image/media/image5.png)',
 '[＞]{.underline}（）',
 '【点睛】分数比大小：',
 '（1）分子相同时，分母小的分数大；',
 '（2）分母相同时，分子大的分数大。',
 '3\\. 一团彩带长5米，捆扎一个礼盒需要86厘米，捆扎这样的4个礼盒需要（ ）厘米长的彩带，还剩（ ）厘米长的彩带。',
 '【答案】 (1). 344 (2). 156',
 '【解析】',
 '【分析】用捆扎一个礼盒需要的长度乘礼盒的个数求出需要多少厘米长的彩带；用彩带的全长减去捆扎礼盒用的彩带长度求出还剩多少厘米的彩带。',
 '【详解】86×4＝344（厘米）',
 '5米＝500厘米',
 '500－344＝156（厘米）',
 '【点睛】本题的关键是注意单位的换算。',
 '4\\. 学校报告厅共有18排座位，每排有22个座位，能容纳多少名学生同时就座？三年级有学生193人，四年级有学生176人，估一估，三、四年级的学生同时就座，坐得下吗？你是怎么估的？',
 '【答案】396名；坐得下；把193看做200，把176看做180进行估算',
 '【解析】',
 '【分析】先用每排的座位数乘排数求出能容纳多少学生，然后

In [46]:
# split_str用于进行下一步的分析（有可能这份试卷是有答题区和答案区，也有可能每道题的答案都在题的下方，还有可能根本没有答案）
all_question, split_str = get_all_question(test_text)
print(f"split_str:\n{split_str}")
if split_str in [-1, 0]:
    print("此试卷无答案")
all_question

split_str:
1


['1\\. 口算![](./notebook/image/media/image1.wmf)\n86＋16＝ 61－25＝ 36＋37＝ 33＋24－15＝\n250＋170＝ 17×3＝ 5×16＝ 280－150＋30＝\n123×0＝ 101×5＝\n【答案】102，36，73，42\n420，51，80，160\n0，505，1，\n【解析】\n【分析】\n【详解】略\n二、用心思考，准确填写。\n2\\. 用分数表示涂色部分，并在横线上填写"＞""＝"或"＜"。\n![](./notebook/image/media/image5.png)\n[ ]{.underline} （ ）\n【答案】＞；\n【解析】\n【分析】把15个小蘑菇平均分成5份，1份是3个，涂颜色的是3个正好是1份，是总数。分子相同时，分母小的分数大。\n【详解】![](./notebook/image/media/image5.png)\n[＞]{.underline}（）\n【点睛】分数比大小：\n（1）分子相同时，分母小的分数大；\n（2）分母相同时，分子大的分数大。\n3\\. 一团彩带长5米，捆扎一个礼盒需要86厘米，捆扎这样的4个礼盒需要（ ）厘米长的彩带，还剩（ ）厘米长的彩带。\n【答案】 (1). 344 (2). 156\n【解析】\n【分析】用捆扎一个礼盒需要的长度乘礼盒的个数求出需要多少厘米长的彩带；用彩带的全长减去捆扎礼盒用的彩带长度求出还剩多少厘米的彩带。\n【详解】86×4＝344（厘米）\n5米＝500厘米\n500－344＝156（厘米）\n【点睛】本题的关键是注意单位的换算。\n4\\. 学校报告厅共有18排座位，每排有22个座位，能容纳多少名学生同时就座？三年级有学生193人，四年级有学生176人，估一估，三、四年级的学生同时就座，坐得下吗？你是怎么估的？\n【答案】396名；坐得下；把193看做200，把176看做180进行估算\n【解析】\n【分析】先用每排的座位数乘排数求出能容纳多少学生，然后将三、四年级的学生人数估算为整百、整百整十数去进行估算，最后和报告厅可以容纳的学生人数进行比较。\n【详解】18×22＝396 （名）\n答：能容纳396名学生同时就座。\n193＋ 176≈200＋ 180＝380 （人）\n380＜ 396\n答：坐

In [47]:
if split_str == 1:
    for row in align_answers_in_questions(all_question):
        print()
        print(row["question"])
        print("-------------------------------------------------------")
        print(f"{row['answer']}")
        print()
        print("============================================================")


1\. 口算![](./notebook/image/media/image1.wmf)
86＋16＝ 61－25＝ 36＋37＝ 33＋24－15＝
250＋170＝ 17×3＝ 5×16＝ 280－150＋30＝
123×0＝ 101×5＝
-------------------------------------------------------
【答案】102，36，73，42
420，51，80，160
0，505，1，
【解析】
【分析】
【详解】略
二、用心思考，准确填写。
2\. 用分数表示涂色部分，并在横线上填写"＞""＝"或"＜"。
![](./notebook/image/media/image5.png)
[ ]{.underline} （ ）
【答案】＞；
【解析】
【分析】把15个小蘑菇平均分成5份，1份是3个，涂颜色的是3个正好是1份，是总数。分子相同时，分母小的分数大。
【详解】![](./notebook/image/media/image5.png)
[＞]{.underline}（）
【点睛】分数比大小：
（1）分子相同时，分母小的分数大；
（2）分母相同时，分子大的分数大。
3\. 一团彩带长5米，捆扎一个礼盒需要86厘米，捆扎这样的4个礼盒需要（ ）厘米长的彩带，还剩（ ）厘米长的彩带。
【答案】 (1). 344 (2). 156
【解析】
【分析】用捆扎一个礼盒需要的长度乘礼盒的个数求出需要多少厘米长的彩带；用彩带的全长减去捆扎礼盒用的彩带长度求出还剩多少厘米的彩带。
【详解】86×4＝344（厘米）
5米＝500厘米
500－344＝156（厘米）
【点睛】本题的关键是注意单位的换算。
4\. 学校报告厅共有18排座位，每排有22个座位，能容纳多少名学生同时就座？三年级有学生193人，四年级有学生176人，估一估，三、四年级的学生同时就座，坐得下吗？你是怎么估的？
【答案】396名；坐得下；把193看做200，把176看做180进行估算
【解析】
【分析】先用每排的座位数乘排数求出能容纳多少学生，然后将三、四年级的学生人数估算为整百、整百整十数去进行估算，最后和报告厅可以容纳的学生人数进行比较。
【详解】18×22＝396 （名）
答：能容纳396名学生同时就座。
193＋ 176≈200＋ 180＝380 （人）

TypeError: 'NoneType' object is not subscriptable

# 寻找答案（试卷存在答题区与答案区）

In [29]:
def get_all_answer_sequence(answer_area_string):
    lines = answer_area_string.splitlines()
    
    inaccuracy_answers = []

    index = find_next_question_index(0, lines)
    while index < len(lines):
#         print(lines[index])
#         print()
        next_index = find_next_question_index(index, lines)
       
        inaccuracy_answers.append("\n".join(lines[index: next_index]))
        index = next_index
#     print("============")
#     for i in inaccuracy_answers:
#         print(i)
#         print()
        
#     print([extract_and_combine_numbers(topic) for topic in inaccuracy_answers])
    inaccuracy_answer_indexes = longest_increasing_subsequence_index(inaccuracy_answers)
   
    processed_inaccuracy_answers = []
    for index, answer_index in enumerate(inaccuracy_answer_indexes):
        if index == len(inaccuracy_answer_indexes) -1 :
            processed_inaccuracy_answers.append(inaccuracy_answers[answer_index])
            break
        processed_inaccuracy_answers.append("\n".join(inaccuracy_answers[answer_index:inaccuracy_answer_indexes[index+1]]))
        
    return refine_answers(processed_inaccuracy_answers)[::-1]

In [30]:
answer_str = generate_answer_area_string(test_text, split_str)

In [31]:
answer_str.splitlines()

['',
 '1.（1）空间的大小 （2）物体的体积 （3）上升 土豆要占有一定的空间',
 '（4）体积（5）',
 '2．(1)A (2)C (3)C A B (4)B来源：www.bcjy123.com/tiku/',
 '3．(1)× (2)√ (3)× (4)× (5)× (6)×',
 '4．不一样大 因为木头比纸要厚一些，在体积一样大的情况下，木盒的容积比纸盒的容积要小一些。',
 '5．有可能 因为每个杯子大小不同，那么4小杯就等于3大杯。',
 '6．略',
 '7．(1)摆成一行时表面积最大，34d㎡；(2)摆成一个大正方体时表面积最小，24d㎡。']

In [32]:
for i in get_all_answer_sequence(answer_area_str_process(answer_str)):
    print(i)
    print()

1.（1）空间的大小 

2．(1)A (2)C (3)C A B (4)B来源：www.bcjy123.com/tiku/

3．(1)× (2)√ (3)× (4)× (5)× (6)×

4．不一样大 因为木头比纸要厚一些，在体积一样大的情况下，木盒的容积比纸盒的容积要小一些。

5．有可能 因为每个杯子大小不同，那么4小杯就等于3大杯。

6．略

7．(1)摆成一行时表面积最大，34d㎡；(2)摆成一个大正方体时表面积最小，24d㎡。



# 对齐

In [33]:
def alignment_answer(all_question, answer_str):
    questions_with_answer = []
    all_answer = get_all_answer_sequence(answer_area_str_process(answer_str))

    questions_map = {extract_and_combine_numbers(question):question for question in reversed(all_question)} 
    answer_map = {extract_and_combine_numbers(answer):answer for answer in reversed(all_answer)}
    
    for sequence_number in questions_map:
        questions_with_answer.append({"question":questions_map.get(sequence_number),"answer":answer_map.get(sequence_number, None)})
    return questions_with_answer

In [34]:
for row in alignment_answer(all_question, answer_str):
    print()
    print(row["question"])
    print("-------------------------------------------------------")
    print(f"{row['answer']}")
    print()
    print("============================================================")


7．把8块棱长1dm的正方体摆成一个长方体。
(1)怎样摆它的表面积最大？是多少平方分米？
来源：www.bcjy123.com/tiku/
(2)怎样摆它的表面积最小，是多少平方分米？

-------------------------------------------------------
7．(1)摆成一行时表面积最大，34d㎡；(2)摆成一个大正方体时表面积最小，24d㎡。


(6)一个长方体纸箱里，放满了苹果，所有苹果的体积就是这个纸箱的容积。( )
4．两个体积一样大的盒子，它们的容积一样大吗？为什么？
![](./notebook/image/media/image1.jpeg)
5．小亮和小茜各有一瓶同样多的饮料，小亮倒了4杯，而小茜却倒了3杯，你认为有可能吗？为什么？
6．动手搭一搭。
用16个大小完全相同的小正方体搭一搭：
(1)搭出两个物体，使它们的体积相同。你能画一画吗？
(2)搭出两个物体，使其中一个物体体积是另一个物体体积的3倍。
-------------------------------------------------------
6．略


(5)把两个完全相同的长方体拼成一个大长方体，体积和表面积都不变。 ( )
-------------------------------------------------------
5．有可能 因为每个杯子大小不同，那么4小杯就等于3大杯。


(4)水杯中装了半杯牛奶，牛奶的体积就是水杯的容积。 ( )
-------------------------------------------------------
4．不一样大 因为木头比纸要厚一些，在体积一样大的情况下，木盒的容积比纸盒的容积要小一些。


(3)所有的物体都有容积。 ( )
-------------------------------------------------------
3．(1)× (2)√ (3)× (4)× (5)× (6)×


(2)所有的物体都有体积。 ( )
-------------------------------------------------------
2．(1)A (2)C (3)C A B (4)B来源：www.bcjy123.com/tiku/


(1

# 生产测试

In [116]:
import pandas as pd
from tqdm import tqdm

# 创建一个空的 DataFrame
df = pd.DataFrame()

# 无答案的文件名列表
failed_files = []

for examination_paper in tqdm(examination_paper_list):
   
    file_path = str(examination_paper["file_path"])
    file_name = file_path[file_path.rfind("/")+1:]

    text = examination_paper["text"]
    all_question, split_str = get_all_question(text)

    if split_str in [-1, 0]:
#             print(f"{file_name} 试卷无答案")
        failed_files.append(file_name)
        continue
        
    if not all_question:
        continue
        
    # question_with_answer like is [{"question":str, "answer":str}]
    if split_str == 1:
        question_with_answer = align_answers_in_questions(all_question)
    else:
        answer_str = generate_answer_area_string(text, split_str)
        question_with_answer = alignment_answer(all_question, answer_str)

    # 创建一个临时的 DataFrame
    temp_df = pd.DataFrame(question_with_answer)

    # 添加文件名列
    temp_df['file_name'] = file_name

    # 使用 pd.concat 将 temp_df 添加到主 DataFrame 中
    df = pd.concat([df, temp_df], ignore_index=True)
   

 12%|█████████▌                                                                    | 199/1615 [00:00<00:01, 888.35it/s]


AttributeError: 'NoneType' object has no attribute 'keys'

In [20]:
# df
df[df['answer'].notna()]

,question,answer,file_name
0,2、同分母分数相加减，分母不变，分子相加减。 （ ）,2、﹥ ﹥ = = ﹥ ﹤ ﹥ ﹥,北师大版小学三年级下册数学第六单元《认识分数》单元测试3（附答案）.md
1,3、＋ = （ ）\n![](./notebook/image/media/image7.j...,3、 1 0,北师大版小学三年级下册数学第六单元《认识分数》单元测试3（附答案）.md
3,1．7的相反数是（）\nA．7 B．﹣7 C．![](./notebook/image/me...,1．7的相反数是（）\nA．7 B．﹣7 C．![](./notebook/image/me...,广西贵港市2017年中考数学试题（word版%2C含解析）.md
4,2．数据3，2，4，2，5，3，2的中位数和众数分别是（）\nA．2，3 B．4，2 C．3...,2．数据3，2，4，2，5，3，2的中位数和众数分别是（）\nA．2，3 B．4，2 C．3...,广西贵港市2017年中考数学试题（word版%2C含解析）.md
5,3．如图是一个空心圆柱体，它的左视图是（）\n![](./notebook/image/me...,3．如图是一个空心圆柱体，它的左视图是（）\n![](./notebook/image/me...,广西贵港市2017年中考数学试题（word版%2C含解析）.md
...,...,...,...
26819,19．2018年1月26日，甘肃省人民政府办公厅发布《甘肃省关于餐饮业质量安全提升工程的实施...,19．解：（1）设表示所抽取3个中有所大学食堂评分不低于9分，至多有1个评分不低于9分记为事...,【全国百强校word】河北省衡水中学2018届高三考前适应性训练6月1日第3天数学（理）试题.md
26820,20．椭圆的左、右焦点为，离心率为，已知过轴上一点作一条直线：，交椭圆于两点，且的周长最大值...,20．解：（1）由题意得，\n∴\n∵，∴，∴，\n∴所求椭圆方程为.\n（2）设，联立得，...,【全国百强校word】河北省衡水中学2018届高三考前适应性训练6月1日第3天数学（理）试题.md
26821,21．已知函数.\n（1）若曲线在处的切线与直线垂直，求实数的值；\n（2）设，若对任意两个...,21．解：（1）\n由题意得，解得\n（2）\n对任意两个不等的正数，恒成立，\n令，则，即...,【全国百强校word】河北省衡水中学2018届高三考前适应性训练6月1日第3天数学（理）试题.md
26822,22．选修4-4：坐标系与参数方程\n在直角坐标系中，曲线的参数方程为（其中为参数，），以坐...,22．解：（1）由题意可得直线和圆的直角坐标方程分别为，\n∵，∴直线过圆的圆心，∴.\n（...,【全国百强校word】河北省衡水中学2018届高三考前适应性训练6月1日第3天数学（理）试题.md


In [21]:
for index, row in df[df['answer'].notna()].iloc[1000:2000].iterrows():
    print()
    print(row["question"])
    print("-------------------------------------------------------")
    print(f"{row['answer']}")
    print()
    print("============================================================")


（6）一个两位数的十位上的数字是![](./notebook/image/media/image1.png)1，个位上的数字是7，这个数是（ ）。
-------------------------------------------------------
（6）17


（7）16前面的一个数是（ ），后面的一个数是（ ）。
\[来源:Z\xx\k.Com\]

-------------------------------------------------------
（7）15 17


1．甲数除以乙数的商是5，甲数与乙数的比是( )，甲数与甲、乙两数之和的比是( )。
-------------------------------------------------------
1．5:1 5:6 


1．（3分）（2019•孝感）计算﹣19+20等于（）
A．﹣39 B．﹣1 C．1 D．39
-------------------------------------------------------
1．（3分）（2019•孝感）计算﹣19+20等于（）
A．﹣39 B．﹣1 C．1 D．39
【考点】有理数的加法．菁优网版权所有
【分析】直接利用有理数的加减运算法则计算得出答案．
【解答】解：﹣19+20＝1．
故选：C．


2．（3分）（2019•孝感）如图，直线l~1~∥l~2~，直线l~3~与l~1~，l~2~分别交于点A，C，BC⊥l~3~交l~1~于点B，若∠1＝70°，则∠2的度数为（）
![](./notebook/image/media/image1.png)
A．10° B．20° C．30° D．40°
-------------------------------------------------------
2．（3分）（2019•孝感）如图，直线l~1~∥l~2~，直线l~3~与l~1~，l~2~分别交于点A，C，BC⊥l~3~交l~1~于点B，若∠1＝70°，则∠2的度数为（）
![](./notebook/image/media/image42.png)
A．10° B．20° C．30° D．40°
【考点】垂线；平行线的性质．菁优网版权所有
【分析】根据平行线的性质和垂直的定义解答即可．
【解答】解：∵

In [18]:
len(failed_files) / len(examination_paper_list)

0.09659442724458205